## Stammzelltisch 16.4.2024 - Large language model tutorial

## Background
* There are a couple of different large language models available
    * OpenAI models
        * GPT-3.5
        * GPT-4.0
    * LLama2
        * 7, 13, 70 billion parameters
        * Here 7 billion parameters, 4-bits --> can be run on laptop with 8 GBs
    * Gemma
    * ...

### Initialize large language model
* Important: Ollama server must be running in the background - can be started in a terminal:
```
ollama serve
```

In [17]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")

### Query large language model

In [18]:
print(llm.invoke("Please show me a recipe for a vegan Tiramisu!"))


Tiramisu is a classic Italian dessert made with ladyfingers soaked in coffee and liqueur, layered with a creamy mascarpone cheese mixture. While traditional Tiramisu requires dairy products, there are plenty of vegan alternatives available that can help you create a delicious and cruelty-free version of this beloved dessert. Here's a recipe for a vegan Tiramisu:

Ingredients:

* 12-16 ladyfingers (you can use any type of unflavored gelatin-based cookies, such as Savoiardi or Baci)
* 1 cup strong brewed coffee (or espresso)
* 2 tablespoons vegan liqueur (such as Kahlúa or rum)
* 8 ounces cashew cream (see note)
* 1/2 cup nut-based cream cheese (such as Tofutti or Kite Hill)
* 1 teaspoon vanilla extract
* 1/4 teaspoon salt
* Cocoa powder for dusting (optional)

Instructions:

1. In a large mixing bowl, combine the coffee and vegan liqueur. Stir until well combined.
2. Dip each ladyfinger into the coffee mixture for about 3-5 seconds on each side. They should be soft and pliable but not 

### Modify the behavior by modifying the prompt

In [19]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You talk like an Australian guy."),
    ("user", "{input}")
])

chain = prompt | llm 
print(chain.invoke({"input": "What is life all about?"}))

G'day mate! Life, eh? Well, ya know what? It's a ripper of a question, innit? *adjusts sunglasses*

Life's like a bloody great adventure, mate. Full of ups and downs, twists and turns. Ya never know what's comin' next, do ya? One minute ya gotta deal with a bit of a hangover after a night on the beer, then the next minute ya could be chasin' after a bloody great croc in the outback. *winks*

But ya know what? That's what makes life so bloody good! It keeps ya on yer toes, mate. Ya gotta keep ya wits about ya and never stop explorin', never stop chasin' after that next great adventure. Life's too short to be bored, innit? *cracks knuckles*

Now, I know some people might say life's all about findin' yer purpose or followin' yer dreams, but mate, I reckon it's more about just enjoyin' the ride. Ya gotta take the good with the bad, and never forget to have a bloody great time while ya can! *takes a swig of beer*

So there ya have it, mate. That's my two bob's worth on what life's all about

### Test what the llm knows about stem cell biology

In [20]:
print(llm.invoke("Please explain the concept of stemness of stem cells to me!"))


Stemness is a term used to describe the characteristic properties and behaviors of stem cells. Stem cells are undifferentiated cells that have the ability to develop into different types of cells in the body, such as blood cells, nerve cells, or muscle cells. The concept of stemness refers to the intrinsic properties and capabilities of stem cells that allow them to maintain their self-renewal capacity and multipotency.

There are two main types of stem cells: embryonic stem cells and adult stem cells. Embryonic stem cells are found in embryos and have the ability to develop into any cell type in the body, while adult stem cells are found in adult tissues and are limited to developing into specific cell types.

The concept of stemness is important because it highlights the unique properties of stem cells that make them different from other cells in the body. These properties include:

1. Self-renewal capacity: Stem cells can divide and give rise to more stem cells, allowing them to ma

In [21]:
print(llm.invoke("Please explain the concept of within-tissue plasticity to me!"))

## Use retrieval augmented generation

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://pubmed.ncbi.nlm.nih.gov/12160836/")
docs = loader.load()

In [ ]:
from langchain_community.embeddings import OllamaEmbeddings
embeddings = OllamaEmbeddings()

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [ ]:
from langchain_core.documents import Document
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

response = retrieval_chain.invoke({"input": "Please explain the concept of stemness to me!"})
print(response["answer"])


## Use RAG for getting information about a specific article

In [ ]:
result = llm.invoke("What did Lutz Leichsenring tell the German broadcaster DW?")
print(result)


In [ ]:
loader = WebBaseLoader("https://www.theguardian.com/world/2024/mar/15/berlins-techno-scene-added-to-unesco-intangible-cultural-heritage-list")
docs = loader.load()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [ ]:
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

response = retrieval_chain.invoke({"input": "What did Lutz Leichsenring tell the German broadcaster DW?"})
print(response["answer"])
